## Q1. Running Elastic

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

In [1]:
!curl localhost:9200

{
  "name" : "3c7b6c8e93e8",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "GeAlxVtrTWmPYegsSUILEA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


What's the version.build_hash value?

### Answer

"build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"

## Getting the data

In [3]:
# !pip install requests

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [1]:
# !pip install elasticsearch

In [3]:
from tqdm.auto import tqdm
import elasticsearch
from elasticsearch import Elasticsearch

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def config_elastic_search():
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    return index_settings


# https://techoverflow.net/2021/08/04/how-to-fix-elasticsearch-exceptions-requesterror-requesterror400-resource_already_exists_exception-index-already-exists-in-python/
def es_create_index_if_not_exists(es, index_name, body, documents):
    """Create the given ElasticSearch index and ignore error if it already exists"""
    try:
        es.indices.create(index=index_name, body=body)
        for doc in tqdm(documents):
            es.index(index=index_name, document=doc)
    except elasticsearch.exceptions.RequestError as ex:
        if ex.error == 'resource_already_exists_exception':
            pass # Index already exists. Ignore.
        else: # Other exception - raise it
            raise ex

def build_elastic_search(elasticsearch_url, documents, index_name="course-questions"):
    index_settings = config_elastic_search()
    es_client = Elasticsearch(elasticsearch_url) 
    # es_client.indices.create(index=index_name, body=index_settings)
    es_create_index_if_not_exists(es=es_client, index_name=index_name, 
                                  body=index_settings, documents=documents)
    return es_client



In [9]:
%%time
elasticsearch_url = 'http://localhost:9200'
index_name = "hw1_elastic_index_name"
es_client = build_elastic_search(elasticsearch_url, documents, index_name)

100%|██████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.18it/s]

CPU times: user 1.05 s, sys: 94.6 ms, total: 1.15 s
Wall time: 23 s


### Answer

index

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [57]:
def build_search_query(num_results, query, text_boost_fields, query_type, filter_dict):
    search_query = {
        "size": num_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": text_boost_fields,
                        "type": query_type
                    }
                }
            }
        }
    }

    return search_query

def elastic_search(index_name, elastic_query, es_client):
    response = es_client.search(index=index_name, body=elastic_query)
    result_docs = []
    result_docs_score = []
    for hit in tqdm(response['hits']['hits']):
        result_docs.append(hit['_source'])
        result_docs_score.append(hit['_score'])
    return result_docs, result_docs_score

In [58]:
query = 'How do I execute a command in a running docker container?'

num_results = 5
text_boost_fields = ["question^4", "text"]
query_type = "best_fields"
# keyword_fields = ["course"]
# filter_dict = {'course': 'data-engineering-zoomcamp'}
elastic_query = build_search_query(num_results=num_results, query=query, 
                                       text_boost_fields=text_boost_fields,
                                       query_type=query_type,filter_dict=filter_dict)
elastic_results, result_docs_score = elastic_search(index_name, elastic_query, es_client)


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 95325.09it/s]


In [59]:
result_docs_score

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

### Answer

84.05

## Q4. Filtering


Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [119]:
def build_search_query(num_results, query, text_boost_fields, query_type, filter_dict):
    search_query = {
        "size": num_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": text_boost_fields,
                        "type": query_type
                    }
                },
                "filter": {
                    "term": filter_dict
                }
            }
        }
    }

    return search_query

In [120]:
query = 'How do I execute a command in a running docker container?'

num_results = 3
text_boost_fields = ["question^4", "text"]
query_type = "best_fields"
# keyword_fields = ["course"]
filter_dict = {'course': 'machine-learning-zoomcamp'}
elastic_query = build_search_query(num_results=num_results, query=query, 
                                       text_boost_fields=text_boost_fields,
                                       query_type=query_type,filter_dict=filter_dict)
elastic_results, result_docs_score = elastic_search(index_name, elastic_query, es_client)


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 36157.79it/s]


In [121]:
elastic_results[0]['question']

'How do I debug a docker container?'

In [122]:
elastic_results[1]['question']

'How do I copy files from my local machine to docker container?'

In [123]:
elastic_results[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Answer

3. 'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [137]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()
    
    context = ""
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [138]:
prompt = build_prompt(query=query, search_results=elastic_results)

In [139]:
len(prompt)

1463

### Answer

1462

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)

In [140]:
# !pip install tiktoken

In [141]:
import tiktoken

In [142]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [143]:
len(encoding.encode(prompt))

323

### Answer

322

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [144]:
from openai import OpenAI


In [146]:
def build_llm(base_url, api_key):
    client = OpenAI(
        base_url=base_url,
        api_key=api_key
    )
    return client


def query_llm(prompt, client, model_name):
    response = client.chat.completions.create(
        model=model_name,
        messages=[{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content


In [147]:
base_url = 'http://localhost:11434/v1/'
api_key = 'ollama'
model_name = 'phi3'
phi3_client = build_llm(base_url, api_key)
response_res = query_llm(prompt=prompt, client=phi3_client, model_name=model_name)

In [148]:
response_res

" To execute a command in a running docker container, you can use the `docker exec` command followed by the `-it` flag for interactive mode and the specific container ID. For example:\n\n```bash\ndocker exec -it <container-id> <command>\n```\n\nReplace `<container-id>` with your actual container's unique identifier (found using `docker ps`) and `<command>` with the command you wish to execute within the container."

### Answer

" To execute a command in a running docker container, you can use the `docker exec` command followed by the `-it` flag for interactive mode and the specific container ID. For example:\n\n```bash\ndocker exec -it <container-id> <command>\n```\n\nReplace `<container-id>` with your actual container's unique identifier (found using `docker ps`) and `<command>` with the command you wish to execute within the container."

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens

* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [155]:
def cal_price(num_in, num_out, num_req):
    in_price_ratio = 0.005/1000
    out_price_ratio = 0.015/1000
    total = num_req*(in_price_ratio*num_in + out_price_ratio*num_out)
    print("total: $", total)

### Answer

In [156]:
cal_price(num_in=150, num_out=250, num_req=1000)

total: $ 4.5


In [157]:
num_prompt_tokens = len(encoding.encode(prompt))
num_res_tokens = len(encoding.encode(response_res))
cal_price(num_in=num_prompt_tokens, num_out=num_res_tokens, num_req=1)

total: $ 0.00295


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw1

* It's possible that your answers won't match exactly. If it's the case, select the closest one.